In [ ]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 51.5 MB/s eta 0:00:00


In [ ]:
import pytorch_lightning as pl
import os
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import torch
import json
from torch.utils.data import TensorDataset, random_split
from transformers.optimization import AdamW
from pytorch_lightning.callbacks import Callback
from tqdm import tqdm

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained("google-t5/t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
class BiaisCorrector(pl.LightningModule):
    def __init__(self):
        super().__init__()
        model_name = 't5-base'
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer = T5TokenizerFast.from_pretrained(model_name)
        self.batch_size = 16
        self.lr = 4e-5

    def encode_text(self, data_path):
        with open(data_path, 'r', encoding='utf-8') as r:
            data = json.load(r)
        for item in tqdm(data):
            source = self.tokenizer("Removes biais :" + item['Source'], max_length=30, truncation=True, padding='max_length', return_tensors='pt')
            target = self.tokenizer(item['Target'], max_length=30, truncation=True, padding='max_length', return_tensors='pt')
            yield source['input_ids'], target['input_ids']

    def to_tensor(self, source_ids, target_ids):
        source_ids = torch.cat(source_ids, dim=0)
        target_ids = torch.cat(target_ids, dim=0)
        data = TensorDataset(source_ids, target_ids)
        return random_split(data, [len(data), 0])[0]

    def prepare_data(self):
        source_ids, target_ids = list(zip(*tuple(self.encode_text('data_base_story.json'))))
        self.train_ds = self.to_tensor(source_ids, target_ids)

        source_ids, target_ids = list(zip(*tuple(self.encode_text('data_base_test.json'))))
        self.test_ds = self.to_tensor(source_ids, target_ids)

    def forward(self, batch, batch_idx):
        source_ids, target_ids = batch[:2]
        return self.model(input_ids=source_ids, labels=target_ids)

    def training_step(self, batch, batch_idx):
        loss = self(batch, batch_idx)[0]
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self(batch, batch_idx)[0]
        self.log('val_loss', loss)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.batch_size, drop_last=True, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.test_ds, batch_size=self.batch_size, drop_last=False, shuffle=False, num_workers=0)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

    def evaluate(self, dataloader, dataset_name):
        self.model.eval()
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batch in dataloader:
                source_ids, target_ids = batch[:2]
                outputs = self.model(input_ids=source_ids, labels=target_ids)
                loss = outputs.loss
                total_loss += loss.item()
                num_batches += 1

        avg_loss = total_loss / num_batches
        print(f"{dataset_name} Loss: {avg_loss:.4f}")
        return avg_loss

trainer = pl.Trainer(
    default_root_dir='logs',
    min_epochs=15,
    max_epochs=20,
    val_check_interval=0.5,
    logger=pl.loggers.TensorBoardLogger('logs/', name='biais_corrector', version=0)
)


para_model = BiaisCorrector()
trainer.fit(para_model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
100%|██████████| 199/199 [00:00<00:00, 2508.93it/s]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory logs/biais_corrector/version_0/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
--------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
train_loader = para_model.train_dataloader()
train_loss = para_model.evaluate(train_loader, "Training Set")

# Évaluer les performances sur l'ensemble de test
test_loader = para_model.val_dataloader()
test_loss = para_model.evaluate(test_loader, "Testing Set")

Training Set Loss: 0.0009
Testing Set Loss: 0.1330


In [ ]:
tokenizer = T5TokenizerFast.from_pretrained("google-t5/t5-base")
input_ids = tokenizer("Removes biais : He emerged from the smoky building, a fireman with soot-streaked gear and unwavering resolve, the true embodiment of bravery.", return_tensors="pt").input_ids
outputs = para_model.model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

True embodiment of bravery.


In [ ]:
torch.save(para_model.state_dict(), 'biais_corrector_with_evaluate.pt')

In [ ]:
# tokenizer = T5TokenizerFast.from_pretrained("google-t5/t5-base")

# model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")

input_ids = tokenizer("Removes biais : She works in an office as a receptionist, answering phone calls and greeting visitors.", return_tensors="pt").input_ids

outputs = para_model.model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

She works in an office as a receptionist, answering phone calls and greeting visitors.


Idée => on donne la phrase d'entrée à un autre modèle et on le force à process tous ses mots, et on multiplie les pobabilités de chaque mot. Le modèle doit prédire cette valeur, en plus d'une nouvelle phrase débiaisée. On fait la différence de corrélation entre homme et femme. La différence de proba que cette phrase arrive. On obtient une mesure du biais en fonction de la phrase.
Que faire de cette mesure de biais ? Ca peut servir de loss?

In [ ]:
# Save the model after training
para_model.model.save_pretrained('biais_corrector_model')
para_model.tokenizer.save_pretrained('biais_corrector_model')

('biais_corrector_model/tokenizer_config.json',
 'biais_corrector_model/special_tokens_map.json',
 'biais_corrector_model/spiece.model',
 'biais_corrector_model/added_tokens.json',
 'biais_corrector_model/tokenizer.json')

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('biais_corrector_model')
tokenizer = T5TokenizerFast.from_pretrained('biais_corrector_model')

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory biais_corrector_model.

In [ ]:
model = torch.load("biais_corrector.pt")

<ipython-input-7-8fb2471ce469>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("biais_corrector.pt")


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
para_model = BiaisCorrector()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:

trainer = pl.Trainer(
    default_root_dir='logs',
    min_epochs=15,
    max_epochs=20,
    val_check_interval=0.5,
    logger=pl.loggers.TensorBoardLogger('logs/', name='biais_corrector', version=0)
)

trainer.fit(para_model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
100%|██████████| 386/386 [00:00<00:00, 2616.28it/s]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M  | eval
------------------------------------------------------------
222 M     Trainable params
0   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
